In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
andrew_gina_docs = SimpleDirectoryReader(input_files=["./assets/sleep/115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt"], filename_as_id=True).load_data()
vector_index = VectorStoreIndex.from_documents(andrew_gina_docs)

2025-10-05 13:49:50,016 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [4]:
vector_engine = vector_index.as_chat_engine()

In [8]:
%pip install llama-index --upgrade

from llama_index.core import ListIndex
from llama_index.core.node_parser import SimpleNodeParser

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)
nodes = node_parser.get_nodes_from_documents(andrew_gina_docs)
list_index = ListIndex(nodes)

Note: you may need to restart the kernel to use updated packages.


In [11]:
list_query_engine = list_index.as_query_engine(
    response_mode = "tree_summarize", use_async = True
)

In [13]:
%pip install llama-index --upgrade

from llama_index.core.tools.query_engine import QueryEngineTool

list_tool = QueryEngineTool.from_defaults(
    query_engine = list_query_engine,
    description="Useful for summarisation of the podcast about sleep and memory with dr. Gina Poe"
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_engine,
    description="Useful fro retrieving of specific content about sleep and memory in the podcast topic"
)

Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install llama-index --upgrade

from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine import RouterQueryEngine 

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ]
)

Note: you may need to restart the kernel to use updated packages.


In [21]:
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ] 
)

In [23]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [24]:
openai.log = "none"

In [30]:
%pip install llama-index --upgrade

from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("Give me a quick summary of the Andrew Huberman podcast with dr. Gina Poe.")
pprint_response(response, show_source=True)

Note: you may need to restart the kernel to use updated packages.


2025-10-05 14:00:27,265 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-05 14:00:27,268 - INFO - Selecting query engine 0: Useful for summarisation of the podcast about sleep and memory with dr. Gina Poe.
2025-10-05 14:00:27,780 - INFO - Retrying request to /chat/completions in 0.479055 seconds
2025-10-05 14:00:29,484 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-05 14:00:31,144 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-05 14:00:31,362 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-05 14:00:33,223 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Final Response: The podcast episode with Dr. Gina Poe on the Huberman
Lab podcast explored the intricate relationship between sleep,
learning, memory, and emotional states. Dr. Poe discussed the impact
of specific sleep phases on learning and memory, the importance of
consistent bedtimes for growth hormone release, the significance of
REM sleep for emotional processing, and the role of the locus ceruleus
in regulating attention and memory. Additionally, they touched on
topics such as sleep spindles in memory consolidation, the therapeutic
aspects of lucid dreaming in trauma recovery, and the importance of
calming the sympathetic nervous system before sleep for adaptive sleep
processes.
______________________________________________________________________
Source Node 1/37
Node ID: 70bc4b82-9ea2-4265-b996-81e30c3bd2d0
Similarity: None
Text: Welcome to the Huberman Lab podcast, where we discuss science
and science-based tools for everyday life, I'm Andrew Huberman and I'm
a professor of 

In [31]:
import nest_asyncio
nest_asyncio.apply()